**Loading Data and creating benchmark model**

In [ ]:
# Defining the path to the Github repository
  file_url = 'https://raw.githubusercontent.com/sedeba19/Chapter-17/main/data_source/Datasets_bank-full.csv'

In [4]:
# Loading data using pandas
import pandas as pd
bankData = pd.read_csv(file_url,sep=";")
bankData.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [ ]:
# Removing the target variable
Y = bankData.pop('y')

In [ ]:
from sklearn.model_selection import train_test_split

# Splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(bankData, Y, test_size=0.3, random_state=123)

In [ ]:
# Using pipeline to transform categorical variable and numeric variables
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder

categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])



In [ ]:
# Defining data types for numeric and categorical features
numeric_features = bankData.select_dtypes(include=['int64', 'float64']).columns
categorical_features = bankData.select_dtypes(include=['object']).columns



In [ ]:
# Defining preprocessor
from sklearn.compose import ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

In [ ]:
# Defining the estimator for processing and classification
from sklearn.linear_model import LogisticRegression
estimator = Pipeline(steps=[('preprocessor', preprocessor),                      
                           ('classifier',LogisticRegression(random_state=123))])


In [ ]:
# Fit the estimator on the training set
estimator.fit(X_train, y_train)  
print("model score: %.2f" % estimator.score(X_test, y_test)) 

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


model score: 0.90


In [ ]:
# Predict on the test set
pred = estimator.predict(X_test)

In [ ]:
# Generating classification report
from sklearn.metrics import classification_report

print(classification_report(pred,y_test))

              precision    recall  f1-score   support

          no       0.98      0.92      0.95     12765
         yes       0.32      0.64      0.43       799

    accuracy                           0.90     13564
   macro avg       0.65      0.78      0.69     13564
weighted avg       0.94      0.90      0.92     13564



**Establishing entities and relationship**

In [ ]:
# Creating the Ids for Demographic Entity
bankData['custID'] = bankData.index.values

bankData['custID'] = 'cust' + bankData['custID'].astype(str)

In [ ]:
# Creating AssetId
bankData['AssetId'] = 0
bankData.loc[bankData.housing == 'yes','AssetId']= 1

In [ ]:
# Creating LoanId
bankData['LoanId'] = 0
bankData.loc[bankData.loan == 'yes','LoanId']= 1

In [ ]:
# Creating Financial behaviour ID
bankData['FinbehId'] = 0
bankData.loc[bankData.default == 'yes','FinbehId']= 1

In [ ]:
# Importing necessary libraries
import featuretools as ft
import numpy as np

In [ ]:
# creating the entity set 'Bankentities'
Bankentities = ft.EntitySet(id = 'Bank')

In [ ]:
# Mapping a dataframe to the entityset to form the parent entity
Bankentities.entity_from_dataframe(entity_id = 'Demographic Data', dataframe = bankData, index = 'custID')

Entityset: Bank
  Entities:
    Demographic Data [Rows: 45211, Columns: 20]
  Relationships:
    No relationships

In [ ]:
# Mapping to parent entity and setting the relationship
Bankentities.normalize_entity(base_entity_id='Demographic Data', new_entity_id='Assets', index = 'AssetId', 
additional_variables = ['housing'])

Bankentities.normalize_entity(base_entity_id='Demographic Data', new_entity_id='Liability', index = 'LoanId', 
additional_variables = ['loan'])

Bankentities.normalize_entity(base_entity_id='Demographic Data', new_entity_id='FinBehaviour', index = 'FinbehId', 
additional_variables = ['default'])


Entityset: Bank
  Entities:
    Demographic Data [Rows: 45211, Columns: 17]
    Assets [Rows: 2, Columns: 2]
    Liability [Rows: 2, Columns: 2]
    FinBehaviour [Rows: 2, Columns: 2]
  Relationships:
    Demographic Data.AssetId -> Assets.AssetId
    Demographic Data.LoanId -> Liability.LoanId
    Demographic Data.FinbehId -> FinBehaviour.FinbehId

**Feature Engineering**

In [ ]:
# Creating aggregation and transformation primitives
aggPrimitives=[
        'std', 'min', 'max', 'mean', 
         'last', 'count'
        
]
tranPrimitives=[
        'percentile', 
         'subtract', 'divide']

In [ ]:
# Defining the new set of features
feature_set, feature_names = ft.dfs(entityset=Bankentities, 
target_entity = 'Demographic Data',
agg_primitives=aggPrimitives,
trans_primitives=tranPrimitives, 
max_depth = 2, 
verbose = 1, 
n_jobs = 1)

Built 3420 features
Elapsed: 01:42 | Remaining: 00:00 | Progress: 100%|██████████| Calculated: 11/11 chunks


In [ ]:
# Reindexing the feature_set
feature_set = feature_set.reindex(index=bankData['custID'])
feature_set = feature_set.reset_index()

In [ ]:
# Displaying the feature set 
feature_set.shape

(45211, 3421)

**Cleaning na values and infinity values**

In [ ]:
# Dropping all Ids
X = feature_set[feature_set.columns[~feature_set.columns.str.contains(
    'custID|AssetId|LoanId|FinbehId')]]


(45211, 3411)

In [ ]:
# Replacing all columns with infinity with nan
X = X.replace([np.inf, -np.inf], np.nan)



(45211, 3411)

In [ ]:
# Dropping all columns with nan
X = X.dropna(axis=1, how='any')
X.shape

(45211, 1046)

**Modelling phase**

In [ ]:
# Splitting train and test sets
from sklearn.model_selection import train_test_split

# Splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=123)

In [ ]:
# Creating the preprocessing pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder

categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])

numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

from sklearn.compose import ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

In [ ]:
# Creating the estimator function and fitting the training set
estimator = Pipeline(steps=[('preprocessor', preprocessor),                      
                           ('classifier',LogisticRegression(random_state=123))])
estimator.fit(X_train, y_train)  
print("model score: %.2f" % estimator.score(X_test, y_test)) 

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


model score: 0.90


In [ ]:
# Predicting on the test set
pred = estimator.predict(X_test)

In [ ]:
# Generating the classification report
from sklearn.metrics import classification_report

print(classification_report(pred,y_test))

              precision    recall  f1-score   support

          no       0.97      0.92      0.95     12716
         yes       0.35      0.65      0.45       848

    accuracy                           0.90     13564
   macro avg       0.66      0.78      0.70     13564
weighted avg       0.94      0.90      0.92     13564

